### Урок 3. Парсинг данных. HTML, Beautiful Soap

### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.

In [2]:
from bs4 import BeautifulSoup as bs 
import requests
import pandas as pd

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
             
}
url = 'https://hh.ru/search/vacancy'



In [32]:
def get_html(url, params=''):
    response = requests.get(url, headers=headers, params=params)
    return response

In [36]:
def hh_content(response):
    dom = bs(response, 'lxml')
    items = dom.find_all('div', class_='vacancy-serp-item__layout')
    hh_vacancy = []

    for item in items:
        hh_vacancy.append(
            {
                'site': 'HeadHanter', # Название сайта
                'title': item.find('h3', class_='bloko-header-section-3').get_text(), # Название вакансии
                'link': item.find('a', class_='serp-item__title').get('href'), # Ссылка на вакансию
                'salary': item.find('span', class_='bloko-header-section-3'), # Зарплата
                'organization': item.find('div', class_='vacancy-serp-item-company').get_text() # Название компании
            }
        )
    for i in hh_vacancy:
      
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            salary_list = i['salary'].split(' ')
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1]
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1]
            else:
                i['salary_min'] = salary_list[0]
                i['salary_max'] = salary_list[2]
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
    return hh_vacancy

In [37]:
def parser_hh():
    post = str(input('Введите название вакансии для парсинга: '))
    pages = int(input('Количество страниц для парсинга: '))
    html = get_html(url)
    if html.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'Парсим страницу {page}')
            html = get_html(url, params={'text': post, 'page': page})
            vacancy.extend(hh_content(html.text))
        hh_result = pd.DataFrame(vacancy)
    else:
        hh_result = html.status_code
    return hh_result

In [38]:
df = parser_hh()
df

Введите название вакансии для парсинга: водитель
Количество страниц для парсинга: 1
Парсим страницу 1


,site,title,link,organization,salary_min,salary_max,salary_currency
0,HeadHanter,Водитель категории Е,https://adsrv.hh.ru/click?b=329606&c=30&meta=D...,"МАГНИТ, Розничная сетьОмск",None,100 000,руб.
1,HeadHanter,Водитель на самовал,https://adsrv.hh.ru/click?b=367192&c=30&meta=6...,ООО ТСКТула,80 000,100 000,руб.
2,HeadHanter,Водитель-курьер на автомобиле компании (юг гор...,https://voronezh.hh.ru/vacancy/76640105?from=v...,Орешкофф.рфСанкт-Петербург,50 000,55 000,руб.
3,HeadHanter,Водитель-курьер на личном автомобиле,https://voronezh.hh.ru/vacancy/71302355?from=v...,СберЛогистикаНовосибирск,75 600,98 000,руб.
4,HeadHanter,Курьер-водитель в Москве на доставку легких за...,https://voronezh.hh.ru/vacancy/77051322?from=v...,ООО КурьеристМосква,None,150 000,руб.
5,HeadHanter,Водитель Категории D,https://voronezh.hh.ru/vacancy/76947525?from=v...,ООО Аэросиб СВидное,50 000,60 000,руб.
6,HeadHanter,Водитель на автомобиле компании,https://voronezh.hh.ru/vacancy/72155946?from=v...,"LamodaМосква, Дмитровская и еще 1",75 000,150 000,руб.
7,HeadHanter,Водитель специального грузового автомобиля мус...,https://voronezh.hh.ru/vacancy/66792562?from=v...,УП Филиал №1 Унитарного предприятия Вторичный ...,2 300,None,бел. руб.
8,HeadHanter,Водитель-международник,https://voronezh.hh.ru/vacancy/76084101?from=v...,ООО ИНТЕРТРАНСАВТОМинск,1 800,2 300,EUR
9,HeadHanter,Водитель-международник (стажер),https://voronezh.hh.ru/vacancy/70867763?from=v...,ООО ИНТЕРТРАНСАВТОМинск,1 800,2 500,EUR
